In [3]:
df['step'] = 1577836800 + df['step'] * 3600 * 24
df['step'] = pd.to_datetime(df['step'], unit='s')


In [5]:
def last1DayTransactionCount(x):
    temp = pd.Series(x.index, index = x.step, name='count_1_day').sort_index()
    count_1_day = temp.rolling('1d').count() - 1
    count_1_day.index = temp.values
    x['count_1_day'] = count_1_day.reindex(x.index)
    return x
def last7DaysTransactionCount(x):
    temp = pd.Series(x.index, index = x.step, name='count_7_days').sort_index()
    count_7_days = temp.rolling('7d').count() - 1
    count_7_days.index = temp.values
    x['count_7_days'] = count_7_days.reindex(x.index)
    return x
def last30DaysTransactionCount(x):
    temp = pd.Series(x.index, index = x.step, name='count_30_days').sort_index()
    count_30_days = temp.rolling('30d').count() - 1
    count_30_days.index = temp.values
    x['count_30_days'] = count_30_days.reindex(x.index)
    return x

In [9]:
import pandas as pd
from creditcard_mod import helpers
df = helpers.load_csv("../datasets/data.csv")
df.head(3)

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,1093826151,4,M,'28007',348934600,'28007',transportation,4.55,0
1,0,352968107,2,M,'28007',348934600,'28007',transportation,39.68,0
2,0,2054744914,4,F,'28007',1823072687,'28007',transportation,26.89,0


In [2]:
compute_features = {
    "nbre_trans_7jrs": 7,
    "nbre_trans_15jrs": 15,
    "nbre_trans_21jrs": 21,
    "nbre_trans_30jrs": 30,

    "montant_debit_moyen_par_trans": 1
    
}

In [3]:
import datetime
def create_transaction_historic(row,cols):
    """
        This function will create features for transactions historic
    """
    result = pd.DataFrame(data=[row],columns=cols)
    end_date = row['step']
    #print(type(end_date))
    begin_date = end_date - 30 + 1
    
    transaction_day = row['step']
    id_emeteur = row['customer']
    id_recepteur = row['merchant']
    
    'query_interval = df.query("step < @end_date and step >= @begin_date \
                                        and customer==@id_emeteur")'
    for key, value in compute_features.items():
        if("montant_debit_moyen_par_trans" in key):
            #print("hello from "+key+"\n")
            res = query_interval.query("step >= @begin_date and customer==@id_emeteur")
            result[key] = res['amount'].sum()/max(1,len(res))
            
    return result

In [8]:
data_train = df.groupby('customer').apply(last1DayTransactionCount)
data_train = data_train.groupby('customer').apply(last7DaysTransactionCount)


In [9]:
data_train.head(-5)

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,count_1_day,count_7_days
0,2020-01-01,1093826151,4,M,'28007',348934600,'28007',transportation,4.55,0,0.0,0.0
1,2020-01-01,352968107,2,M,'28007',348934600,'28007',transportation,39.68,0,0.0,0.0
2,2020-01-01,2054744914,4,F,'28007',1823072687,'28007',transportation,26.89,0,0.0,0.0
3,2020-01-01,1760612790,3,M,'28007',348934600,'28007',transportation,17.25,0,0.0,0.0
4,2020-01-01,757503768,5,M,'28007',348934600,'28007',transportation,35.72,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
594633,2020-06-28,1039390058,4,M,'28007',45060432,'28007',hotelservices,190.31,0,1.0,6.0
594634,2020-06-28,728039227,6,M,'28007',348934600,'28007',transportation,27.93,0,0.0,6.0
594635,2020-06-28,728039227,6,M,'28007',1823072687,'28007',transportation,30.04,0,1.0,7.0
594636,2020-06-28,1904086644,5,F,'28007',1823072687,'28007',transportation,28.93,0,0.0,5.0


In [5]:
import os
parent_dir = "PartialResults"
for i in range(8):
    # Path
    directory = "Part_"+str(i)
    path = os.path.join(parent_dir, directory) 
    os.mkdir(path) 
    print("Directory '% s' created" % directory)

FileExistsError: [Errno 17] File exists: 'PartialResults/Part_0'

In [6]:
import shutil
folder = "PartialResults"

for root, subdirs, files in os.walk(folder):
    for d in subdirs:
        if d == ".ipynb_checkpoints":
            shutil.rmtree(os.path.join(root, d))
subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
len(subfolders)

8

In [7]:
%%time
import pandas as pd
import requests
import numpy as np
from tqdm import tqdm
import multiprocessing as mp
tqdm.pandas()

import multiprocessing as mp

#sample_dataset = cleaned_dataset[:15000]


def compute_features_per_split(args):
    print(len(args[0]))
    #result = args[0].apply(complete_trans_infos,axis=1)
    #data = dataset.query("identifier in @args[0]")
    result = args[0].apply(create_transaction_historic,axis=1,cols=args[0].columns)
    result.to_pickle(args[1]+"/checkpoint.pkl")
    print(result.head())
    return result

def parallelize_dataframe(df, func, n_cores=8):
    print("beginning the process...")
    df_split = np.array_split(df, n_cores)
    pool = mp.Pool(n_cores)
    args = zip(df_split,subfolders)
    df = pd.concat(pool.map(func,args ))
    #pool.map(func,args )
    pool.close()
    pool.join()
    print("process done")
    return df

CPU times: user 295 ms, sys: 702 ms, total: 997 ms
Wall time: 230 ms


/home/muhammad/.cache/pypoetry/virtualenvs/fraud-creditcard-ml-ansh3wJ6-py3.8/lib/python3.8/site-packages/tqdm/std.py:701: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
%%time
fraud_dataset = parallelize_dataframe(df[500000:500008], compute_features_per_split)

beginning the process...
1111

In [13]:
fraud_dataset.head()

,0,age,amount,category,customer,fraud,gender,merchant,step,zipMerchant,zipcodeOri
500000,step customer age gender zipcodeOri ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
import pandas as pd
import os
data_directory = "PartialResults/"
def load_data(directoryPath):
    #print(directoryPath)
    result_df = pd.DataFrame()
    for root, subdirs, files in os.walk(directoryPath):
        
        for dir in subdirs:
            print(dir)
            path = root+dir+"/checkpoint.pkl"
            if os.path.exists(path):
                df = pd.read_pickle(path)
                result_df = pd.concat([result_df,df],axis=0)
            
            #print(list(df.values[0]))
            #return result_df,columns
    columns = list(df.values[0])
    return result_df,columns

def get_data(row,data):
    #print(row.values[0])
    data.append(row.values[0])
 
global_result = []
agg_sample,columns = load_data(data_directory)
agg_sample[0].apply(get_data,data=global_result)
columns

Part_0
Part_4
Part_6
Part_2
Part_1
Part_3
Part_7
Part_5


UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
enriched_fraud_dataset = pd.DataFrame(data=global_result,columns=columns)
enriched_fraud_dataset.head()

In [ ]:
enriched_fraud_dataset.shape